In [40]:
from datasets import load_dataset
import random
dataset = load_dataset("Universal-NER/Pile-NER-type")
random_indices = random.sample(range(len(dataset['train'])), 10000)
sampled_data = [dataset['train'][i] for i in random_indices]
print(f"Số lượng mẫu đã chọn ngẫu nhiên: {len(sampled_data)}")


Số lượng mẫu đã chọn ngẫu nhiên: 10000


In [15]:
def convert_to_instruction_format(example):
    instructions = []
    
    conversation = example['conversations']
    text = None
    instruction = None
    
    for turn in conversation:
        if turn['from'] == 'human' and 'Text:' in turn['value']:
            text = turn['value'].replace('Text:', '').strip()
            break
    
    if text:
        for turn in conversation:
            if turn['from'] == 'human' and 'What describes' in turn['value']:
                instruction = turn['value']
            elif turn['from'] == 'gpt' and instruction:
                output = turn['value']
                instructions.append({
                    'instruction': instruction,
                    'input': text,
                    'output': output
                })
                instruction = None
    
    return {"instructions": instructions} if instructions else None


In [16]:
dataset['train']['conversations'][:1]

[[{'from': 'human',
   'value': 'Text: Q:\n\nPosition character based on enemy coordinates in lua\n\nI have written a function here which should turn my character based on enemy coordinates but it\'s not perfect because it does not always turn where I want it to and perhaps there is a better way of writing it\nlocal myPosition = {x = 350, y = 355}\nlocal enemyPosition = {x = 352, y = 354}\nlocal xValue, yValue, xDir, yDir, dir\n\nif myPosition.x > enemyPosition.x then\n    xValue = myPosition.x - enemyPosition.x\nelseif myPosition.x < enemyPosition.x then\n    xValue = myPosition.x - enemyPosition.x\nelse\n    xValue = 0\nend\n\nif myPosition.y > enemyPosition.y then\n    yValue = myPosition.y - enemyPosition.y\nelseif myPosition.y < enemyPosition.y then\n    yValue = myPosition.y - enemyPosition.y\nelse\n    yValue = 0\nend\n\nif xValue < 0 then\n    xDir = "TURN RIGHT"\nelseif xValue > 0 then\n    xDir = "TURN LEFT"\nend\n\nif yValue < 0 then\n    yDir = "TURN DOWN"\nelseif yValue > 

In [17]:
def process_dataset(dataset):
    new_dataset = []
    for example in dataset:
        converted_example = convert_to_instruction_format(example)
        if converted_example:
            new_dataset.append(converted_example)
    return new_dataset

In [43]:
train_data = process_dataset(sampled_data)

In [44]:
train_data[:2]

[{'instructions': [{'instruction': 'What describes Concept in the text?',
    'input': 'Q:\n\nWhy some applications such as Clash of Clans are for android 4.0 to higher? Why some applications such as Clash of Clans are for android 4.0 to higher? While  we can with setting "android:minSdkVersion" equal to 8, we can use from a wide range of devices. Why some manufacturers do not put  "android:minSdkVersion" equal to 8? While this is useful and very easy! A:\n\nWhy some applications such as Clash of Clans are for android 4.0 to higher? Either:\n\nThey need some API or device feature that is only available on Android 4.0 and higher, such as TextureView, or\nThey determined that older devices would not have the system requirements (CPU, GPU, RAM, etc.) that their app needs, or\nThey did not want to spend the effort to test their app older devices, or\nAny other reason, as they have the freedom to choose to support whatever they want\n\nWhile we can with setting "android:minSdkVersion" equal

In [46]:
print(f"Samples: {len(train_data)}")

Samples: 10000


In [56]:
# Chuyển đổi dữ liệu thành định dạng cho mT5
def prepare_mt5_data(data):
    mt5_data = []
    for example in data:
        for instruction_example in example['instructions']:
            instruction = instruction_example['instruction']
            text = instruction_example['input']
            output = instruction_example['output']
            
            # Tạo input string cho mT5
            input_text = f"Instruction: {instruction}\nInput: {text}"
            if isinstance(output, list):
                output_text = ", ".join(map(str, output)) if output else "None"
            else:
                output_text = str(output)
            
            mt5_data.append({'input': input_text, 'output': output_text})
    return mt5_data

# Áp dụng chuyển đổi cho dữ liệu huấn luyện
train_data_mt5 = prepare_mt5_data(train_data)

# Kiểm tra một số mẫu sau khi chuyển đổi
for sample in train_data_mt5[7:8]:
    print(f"Input:\n{sample['input']}")
    print("Output:", sample['output'])
    # print()


Input:
Instruction: What describes hardware in the text?
Input: Q:

Why some applications such as Clash of Clans are for android 4.0 to higher? Why some applications such as Clash of Clans are for android 4.0 to higher? While  we can with setting "android:minSdkVersion" equal to 8, we can use from a wide range of devices. Why some manufacturers do not put  "android:minSdkVersion" equal to 8? While this is useful and very easy! A:

Why some applications such as Clash of Clans are for android 4.0 to higher? Either:

They need some API or device feature that is only available on Android 4.0 and higher, such as TextureView, or
They determined that older devices would not have the system requirements (CPU, GPU, RAM, etc.) that their app needs, or
They did not want to spend the effort to test their app older devices, or
Any other reason, as they have the freedom to choose to support whatever they want

While we can with setting "android:minSdkVersion" equal to 8, we can use from a wide range

In [57]:
import json

# Lưu train_data vào file JSON
with open('train_data.json', 'w', encoding='utf-8') as f:
    json.dump(train_data_mt5, f, ensure_ascii=False, indent=4)

print("Dữ liệu đã được lưu thành công vào file train_data.json")

Dữ liệu đã được lưu thành công vào file train_data.json
